In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 25 14:55:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/drive/MyDrive/Colab_Projects/21_F_딥러닝_김현우/
!ls

/content/drive/MyDrive/Colab_Projects/21_F_딥러닝_김현우
 data			 Datasets	     u.contact	   u.type
 Data_processing.ipynb	 Deep_Learning.pdf   u.platform
'dataset.ipynb의 사본'	 README.md	     u.sentiment


In [ ]:
!pip install transformers

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline

In [ ]:
DATASET_PATH1 = './Datasets/Reviews.csv'
DATASET_PATH2 = './Datasets/product.csv'

#import Review dataset
Amazon_review = pd.read_csv(DATASET_PATH1)
#View current data set
Amazon_review

,Score,Summary,Text
0,5,SPORTea,We have a Michigan-based coffee cafe (now rena...
1,5,Love it...start making bigger boxes!,"I don't drink a lot of flavored k-cups, but pe..."
2,5,Great product - great price,"I don't drink a lot of flavored k-cups, but pe..."
3,5,Great tasting tea,I tried the Frooties mBerry and the effect las...
4,5,sport tea,"Being married to a person from England, as mos..."
...,...,...,...
414822,5,Will not do without,as it should be for $6 a bottle.
414823,2,disappointed,OUR FAVORITE! EXCELLENT ITEM!!
414824,5,Perfect for our maltipoo,It is an excellent product.
414825,5,Favorite Training and reward treat,they are excellent


In [ ]:
product_info = pd.read_csv(DATASET_PATH2)
product_info

,product,site_version,time,title
0,sneakers,desktop,2019-01-11 9:24,banner_click
1,sneakers,desktop,2019-01-09 9:38,banner_show
2,sports_nutrition,desktop,2019-01-09 9:12,banner_show
3,company,desktop,2019-01-03 8:58,banner_show
4,company,desktop,2019-01-03 8:59,banner_click
...,...,...,...,...
1048570,sports_nutrition,mobile,2019-02-01 14:41,banner_click
1048571,sports_nutrition,mobile,2019-01-11 16:41,banner_show
1048572,sports_nutrition,mobile,2019-01-13 8:24,banner_show
1048573,accessories,mobile,2019-01-20 22:45,banner_show


In [ ]:
#Drop all NaN value for integrity
Amazon_review = Amazon_review.drop(['Summary'], axis = 1)

Amazon_review.dropna()
product_info.dropna()

Amazon_review = Amazon_review[:250000]
product_info = product_info[:250000]

Amazon_review

,Score,Text
0,5,We have a Michigan-based coffee cafe (now rena...
1,5,"I don't drink a lot of flavored k-cups, but pe..."
2,5,"I don't drink a lot of flavored k-cups, but pe..."
3,5,I tried the Frooties mBerry and the effect las...
4,5,"Being married to a person from England, as mos..."
...,...,...
249995,5,"My younger lab, Bella, who is 10 months absolu..."
249996,5,Lab enjoys the toy--now. When the rope was att...
249997,5,"I ordered ""Nido"" because my old favorite, ""Mil..."
249998,3,"This is a very tasty, gluten free, corn, soy a..."


In [ ]:
product_info

,product,site_version,time,title
0,sneakers,desktop,2019-01-11 9:24,banner_click
1,sneakers,desktop,2019-01-09 9:38,banner_show
2,sports_nutrition,desktop,2019-01-09 9:12,banner_show
3,company,desktop,2019-01-03 8:58,banner_show
4,company,desktop,2019-01-03 8:59,banner_click
...,...,...,...,...
249995,company,mobile,2019-03-04 4:59,banner_show
249996,sports_nutrition,mobile,2019-02-24 2:42,banner_show
249997,company,mobile,2019-03-06 18:37,banner_show
249998,company,mobile,2019-01-20 9:29,banner_show


In [ ]:
encoder = LabelEncoder()

Cols = ['product', 'site_version', 'title']

for column in Cols:
  product_info[column] = encoder.fit_transform(product_info[column])

product_info = product_info.drop(['time'], axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
product_info

,product,site_version,title
0,3,0,0
1,3,0,1
2,4,0,1
3,2,0,1
4,2,0,0
...,...,...,...
249995,2,1,1
249996,4,1,1
249997,2,1,1
249998,2,1,1


In [ ]:
#Make sure you have GPU
sentiment_analysis = pipeline('sentiment-analysis', device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [ ]:
#Processing text data's sentiment score from our dataset.csv
temp_text = Amazon_review['Text'].tolist()
sentiment = []

for idx ,text in enumerate(temp_text):
  sentiment.append(sentiment_analysis(text)[0])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
sentiment = pd.DataFrame(sentiment)
sentiment

,label,score
0,NEGATIVE,0.998880
1,NEGATIVE,0.679127
2,NEGATIVE,0.679127
3,NEGATIVE,0.993714
4,POSITIVE,0.999573
...,...,...
249995,POSITIVE,0.999212
249996,NEGATIVE,0.997865
249997,POSITIVE,0.997383
249998,POSITIVE,0.999656


In [ ]:
Amazon_review = pd.concat([Amazon_review, sentiment], axis = 1)
Amazon_review = Amazon_review.drop(['Text'], axis = 1)
Amazon_review

,Score,label,score
0,5,NEGATIVE,0.998880
1,5,NEGATIVE,0.679127
2,5,NEGATIVE,0.679127
3,5,NEGATIVE,0.993714
4,5,POSITIVE,0.999573
...,...,...,...
249995,5,POSITIVE,0.999212
249996,5,NEGATIVE,0.997865
249997,5,POSITIVE,0.997383
249998,3,POSITIVE,0.999656


In [ ]:
del[[sentiment]]

In [ ]:
final_dataset = pd.concat([Amazon_review, product_info], axis = 1)
final_dataset

,Score,label,score,product,site_version,title
0,5,NEGATIVE,0.998880,3,0,0
1,5,NEGATIVE,0.679127,3,0,1
2,5,NEGATIVE,0.679127,4,0,1
3,5,NEGATIVE,0.993714,2,0,1
4,5,POSITIVE,0.999573,2,0,0
...,...,...,...,...,...,...
249995,5,POSITIVE,0.999212,2,1,1
249996,5,NEGATIVE,0.997865,4,1,1
249997,5,POSITIVE,0.997383,2,1,1
249998,3,POSITIVE,0.999656,2,1,1


In [ ]:
final_dataset['label'] = encoder.fit_transform(final_dataset['label'])
final_dataset

,Score,label,score,product,site_version,title
0,5,0,0.998880,3,0,0
1,5,0,0.679127,3,0,1
2,5,0,0.679127,4,0,1
3,5,0,0.993714,2,0,1
4,5,1,0.999573,2,0,0
...,...,...,...,...,...,...
249995,5,1,0.999212,2,1,1
249996,5,0,0.997865,4,1,1
249997,5,1,0.997383,2,1,1
249998,3,1,0.999656,2,1,1


In [39]:
import random
user_id = []

for i in range(0, 250000):
  user_id.append(random.randint(1,2500))

In [40]:
user_id = pd.DataFrame(user_id, columns= ['user_id'])
user_id

,user_id
0,138
1,999
2,510
3,1588
4,1912
...,...
249995,2459
249996,2163
249997,585
249998,1394


In [41]:
final_dataset = pd.concat([user_id, final_dataset], axis = 1)
final_dataset

,user_id,Score,label,score,product,site_version,title
0,138,5,0,0.998880,3,0,0
1,999,5,0,0.679127,3,0,1
2,510,5,0,0.679127,4,0,1
3,1588,5,0,0.993714,2,0,1
4,1912,5,1,0.999573,2,0,0
...,...,...,...,...,...,...,...
249995,2459,5,1,0.999212,2,1,1
249996,2163,5,0,0.997865,4,1,1
249997,585,5,1,0.997383,2,1,1
249998,1394,3,1,0.999656,2,1,1


In [43]:
final_dataset.to_csv('u.data', index = False, index_label= ['user_id', 'rate', 'sentiment', 'polarity', 'device', 'method'])

In [30]:
final_dataset = pd.read_csv('u.data')
final_dataset

,Score,label,score,product,site_version,title
0,5,0,0.998880,3,0,0
1,5,0,0.679127,3,0,1
2,5,0,0.679127,4,0,1
3,5,0,0.993714,2,0,1
4,5,1,0.999573,2,0,0
...,...,...,...,...,...,...
249995,5,1,0.999212,2,1,1
249996,5,0,0.997865,4,1,1
249997,5,1,0.997383,2,1,1
249998,3,1,0.999656,2,1,1
